# Import Libraries

In [1]:
from sklearn.cluster import KMeans
from config import *
import pandas as pd
import numpy as np
import random
from collections import defaultdict
import matplotlib.pyplot as plt

NUM_CLUSTERS = 5

# Generate Locations (Demand Point, Charging Points, Depot)

In [2]:
random.seed(SEED)

# Electric Vehicle Points
ev_pts = pd.read_csv("Unique_EV_Points.csv")
charge_pts = ev_pts.sample(n=NUM_CHARGERS)[['Longitude', 'Latitude']] # dataframe
charge_pts_list = list(charge_pts.itertuples(index=False, name=None)) # list

# Delivery locations Points
dl = pd.read_csv("DeliveryLocations.csv")
todays_locations = dl.sample(n=NUM_CUSTOMERS)
dl_list = list(todays_locations.itertuples(index=False, name=None))

# Fedex Ship Centre
fedex_centre = [(104.0023106, 1.3731437)]

locations = fedex_centre + dl_list + charge_pts_list # combined list of all locations
print(locations)

[(104.0023106, 1.3731437), (103.9379441, 1.327974074), (103.766014, 1.322061089), (103.9347819, 1.320439328), (103.7430471, 1.37691702), (103.9082329, 1.387631064), (103.9375717, 1.329973112), (103.9571568, 1.344212748), (103.9101669, 1.398068147), (103.8725936, 1.366958513), (103.7946714, 1.437133553), (103.7647977, 1.314639131), (103.8551037, 1.364541709), (103.8942909, 1.379118191), (103.7002088, 1.342661828), (103.8307906, 1.358081063), (103.8492648, 1.367977051), (103.9310495, 1.349913483), (103.9418509, 1.32643686), (103.9325443, 1.355446818), (103.8206086, 1.446866084), (103.8511342, 1.373405406), (103.8766449, 1.393393644), (103.8456262, 1.336264089), (103.9101808, 1.392335405), (103.8495604, 1.422097317), (103.9452683, 1.358778559), (103.7677633, 1.378312421), (103.9384609, 1.335925042), (103.8998227, 1.393250113), (103.933874, 1.357824582), (103.8170446, 1.446701271), (103.7787005, 1.438126441), (103.83071, 1.426956151), (103.9100225, 1.392092062), (103.9157399, 1.304607086),

# K-means clustering

In [3]:
random.seed(SEED)
X = np.array(dl_list) # convert to numpy array, prepare for clustering
kmeans = KMeans(n_clusters = NUM_CLUSTERS, random_state = SEED)
labels = kmeans.fit_predict(X)

clustered_customers = defaultdict(list)
for idx, label in enumerate(labels):
    clustered_customers[label].append(dl_list[idx])

# Visualise clusters, charging point and depot

In [4]:
import folium
from folium.plugins import MarkerCluster
from matplotlib import cm

# Center the map on Singapore (or your depot)
m = folium.Map(location=fedex_centre[0][::-1], zoom_start=12)  # Note: folium uses (lat, lon)

# Color map for clusters (tab10 is fine for <=10 clusters)
colormap = cm.get_cmap('tab10', NUM_VEHICLES)

# Add clustered demand points
for cluster_id, points in clustered_customers.items():
    color = "#{:02x}{:02x}{:02x}".format(
        int(colormap(cluster_id)[0]*255),
        int(colormap(cluster_id)[1]*255),
        int(colormap(cluster_id)[2]*255)
    )
    for lat, lon in points:
        folium.CircleMarker(
            location=(lon, lat),
            radius=5,
            color=color,
            fill=True,
            fill_opacity=0.8
        ).add_to(m)

# Add depot
folium.Marker(
    location=fedex_centre[0][::-1],
    popup='Depot',
    icon=folium.Icon(color='red', icon='home')
).add_to(m)

# Add charger points
for lat, lon in charge_pts_list:
    folium.Marker(
        location=(lon, lat),
        popup='Charger',
        icon=folium.Icon(color='green', icon='flash')
    ).add_to(m)

# Display map
m

# Assigning vehicles to each cluster

Assign vehicles to clusters proportionally to the number of vehicles in each cluster.

- Count customer in each cluster

- Compute share of each vehicle

- Allocate vehicles using share

- Ensure solution is feasible

## Distance, Energy and Time Matrix

In [5]:
# Total nodes: 1 depot + customers + chargers
n = len(locations)
customer_nodes = list(range(1, 1 + NUM_CUSTOMERS)) # gets the index/nodes of customers
charger_nodes = list(range(1 + NUM_CUSTOMERS, n))  

# Compute Distance & Energy Matrices
distance_matrix = np.zeros((n, n))
time_matrix = np.zeros((n, n))
energy_matrix = np.zeros((n, n))

def calc_dist(p1, p2):
    """
    Calculates the straight-line (euclidean) distance
    """
    lon_diff = p1[0] - p2[0]
    lat_diff = p1[1] - p2[1]
    return 111 * np.sqrt(lon_diff**2 + lat_diff**2)

for i in range(n):
    for j in range(n):
        if i != j:
            dist = calc_dist(locations[i], locations[j])
            distance_matrix[i][j] = dist
            energy_matrix[i][j] = dist * ENERGY_PER_KM
            time_matrix[i][j] = dist * TIME_PER_KM


## Code logic + Constraint Handling

In [10]:
from collections import defaultdict
import numpy as np

vehicle_routes = defaultdict(list)
unassigned_customers = set(customer_nodes)  # start with all customers unassigned
vehicle_id = 0

for cluster_id, cluster_points in clustered_customers.items():
    if vehicle_id >= NUM_VEHICLES:
        break  # ran out of vehicles

    # Convert cluster point (lon, lat) → location indices
    cluster_indices = [locations.index(pt) for pt in cluster_points]
    
    route = [0]  # start at depot
    battery = BATTERY_CAPACITY
    time = 0.0
    used_customers = []

    current_node = 0  # depot index
    for cust_node in cluster_indices:
        if cust_node not in unassigned_customers:
            continue

        e_need = energy_matrix[current_node][cust_node]
        t_need = time_matrix[current_node][cust_node]

        # Battery constraint
        if battery < e_need:
            # find nearest charger
            nearest_charger = min(
                charger_nodes,
                key=lambda c: distance_matrix[current_node][c]
            )
            # insert charger stop
            route.append(nearest_charger)
            battery = BATTERY_CAPACITY
            time += time_matrix[current_node][nearest_charger]
            current_node = nearest_charger

        # Time constraint
        if time + t_need > TOTAL_TIME_ALLOWED:
            continue  # skip this customer for now

        # Assign customer
        route.append(cust_node)
        battery -= e_need
        time += t_need
        current_node = cust_node
        used_customers.append(cust_node)
        unassigned_customers.remove(cust_node)

    # Return to depot
    if current_node != 0:
        time += time_matrix[current_node][0]
        route.append(0)

    vehicle_routes[vehicle_id] = route
    vehicle_id += 1

# Optional: print unassigned customers
if unassigned_customers:
    print("⚠️ Unassigned customers (due to time/battery):", sorted(unassigned_customers))

## View BFS

In [22]:
import folium
from matplotlib import cm

# Create or reuse your base map
m = folium.Map(location=fedex_centre[0][::-1], zoom_start=12)

# Color map for vehicles
colormap = cm.get_cmap('tab10', NUM_VEHICLES)

# Plot each vehicle's route
for veh_id, route in vehicle_routes.items():
    color = "#{:02x}{:02x}{:02x}".format(
        int(colormap(veh_id)[0]*255),
        int(colormap(veh_id)[1]*255),
        int(colormap(veh_id)[2]*255)
    )

    coords = [(locations[i][1], locations[i][0]) for i in route]  # (lat, lon)

    folium.PolyLine(
        coords,
        color=color,
        weight=4,
        opacity=0.8,
        tooltip=f"Vehicle {veh_id}"
    ).add_to(m)

    # Mark start and end points
    folium.Marker(
        coords[0],
        icon=folium.Icon(color='blue', icon='play'),
        popup=f"Start V{veh_id}"
    ).add_to(m)

    folium.Marker(
        coords[-1],
        icon=folium.Icon(color='red', icon='stop'),
        popup=f"End V{veh_id}"
    ).add_to(m)

    # Mark each stop with a circle marker and label
    for idx, node in enumerate(route):
        lat, lon = locations[node][1], locations[node][0]
        folium.CircleMarker(
            location=(lat, lon),
            radius=4,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.9,
            popup=folium.Popup(f"V{veh_id} - Stop {idx}<br>Node {node}", max_width=150),
            tooltip=f"Node {node}"
        ).add_to(m)

# Add depot
folium.Marker(
    location=fedex_centre[0][::-1],
    popup='Depot',
    icon=folium.Icon(color='red', icon='home')
).add_to(m)

# Add charger points
for lat, lon in charge_pts_list:
    folium.Marker(
        location=(lon, lat),
        popup='Charger',
        icon=folium.Icon(color='green', icon='flash')
    ).add_to(m)

# Show the map
m

## Objective Function Value of BFS

In [28]:
total_distance = 0.0
print("\n────────── Objective Function Breakdown ──────────")

print(vehicle_routes)

for veh_id, route in vehicle_routes.items():
    route_distance = 0.0
    for i, j in zip(route, route[1:]):
        d = distance_matrix[i][j]
        route_distance += d
    total_distance += route_distance
    print(f"Vehicle {veh_id}: distance = {route_distance:.2f} km")

print(f"\n✅ Total Objective Value (Distance): {total_distance:.2f} km")


────────── Objective Function Breakdown ──────────
defaultdict(<class 'list'>, {0: [0, 1, 3, 6, 7, 17, 18, 19, 26, 28, 30, 35, 44, 0], 1: [0, 2, 4, 11, 14, 55, 27, 36, 43, 0], 2: [0, 5, 8, 9, 13, 22, 24, 29, 34, 39, 45, 46, 47, 49, 0], 3: [0, 10, 20, 25, 31, 32, 33, 0], 4: [0, 12, 15, 16, 21, 23, 37, 38, 40, 41, 52, 42, 48, 50, 0]})
Vehicle 0: distance = 48.77 km
Vehicle 1: distance = 91.16 km
Vehicle 2: distance = 54.48 km
Vehicle 3: distance = 66.18 km
Vehicle 4: distance = 85.37 km

✅ Total Objective Value (Distance): 345.96 km


In [31]:
print("\n────────── Feasibility & Objective Function Check ──────────")

total_distance = 0.0

for veh_id, route in vehicle_routes.items():
    route_distance = 0.0
    time_accum = 0.0
    battery = BATTERY_CAPACITY
    feasible = True

    print(f"\nVehicle {veh_id}:")
    print(f"{'Leg':<10}{'Dist(km)':>10}{'Time(min)':>12}{'Battery(kWh)':>15}")

    for i, j in zip(route, route[1:]):
        d = distance_matrix[i][j]
        t = time_matrix[i][j]
        e = energy_matrix[i][j]

        # Recharge if leaving charger
        if i in charger_nodes:
            battery = BATTERY_CAPACITY

        # Check battery and time before subtracting
        if battery < e or time_accum + t > TOTAL_TIME_ALLOWED:
            feasible = False

        battery -= e
        time_accum += t
        route_distance += d

        print(f"{f'{i}→{j}':<10}{d:10.2f}{t:12.2f}{battery:15.2f}")

    total_distance += route_distance

    print(f"Total distance: {route_distance:.2f} km")
    print(f"Total time:     {time_accum:.2f} min")
    print(f"Final battery:  {battery:.2f} kWh")
    print(f"{'✅ Feasible' if feasible else '❌ Infeasible'} (time ≤ {TOTAL_TIME_ALLOWED} min, battery ≥ 0)")

print(f"\n✅ Total Objective Value (Distance): {total_distance:.2f} km")


────────── Feasibility & Objective Function Check ──────────

Vehicle 0:
Leg         Dist(km)   Time(min)   Battery(kWh)
0→1             8.73       26.19           8.37
1→3             0.91        2.72           8.20
3→6             1.10        3.31           7.99
6→7             2.69        8.06           7.49
7→17            2.97        8.90           6.93
17→18           2.87        8.61           6.40
18→19           3.38       10.15           5.77
19→26           1.46        4.38           5.49
26→28           2.65        7.94           5.00
28→30           2.48        7.45           4.53
30→35           6.24       18.72           3.37
35→44           7.60       22.79           1.95
44→0            5.70       17.11           0.88
Total distance: 48.77 km
Total time:     146.32 min
Final battery:  0.88 kWh
✅ Feasible (time ≤ 600 min, battery ≥ 0)

Vehicle 1:
Leg         Dist(km)   Time(min)   Battery(kWh)
0→2            26.83       80.50           4.98
2→4             6.60       1